In [3]:
import awkward as ak
import uproot
import numpy as np
from yahist import Hist1D, Hist2D
import numba as nb
import math
# import parquet as pq

In [4]:
import json
with open("/home/users/azecchin/Analysis/ResonantGGTT/Outputs/XtoYH_ggbb_2018_HLT_MCfix_looper/CatOptim/optim_results.json") as f:
    limits_json = json.load(f) 
with open("/home/users/azecchin/Analysis/ResonantGGTT/Outputs/XtoYH_ggbb_2018_noMX/CatOptim/optim_results.json") as f:
    noMX_limits_json = json.load(f) 
with open("/home/users/azecchin/Analysis/ResonantGGTT/Outputs/XtoYH_DDGJets/CatOptim/optim_results.json") as f:
    noMX_DDGJ_limits_json = json.load(f) 

In [5]:
import pandas as pd

## PNN checks

- [ ] Check limits evolution w w/o DataDrivenGJ
- [ ] Check SR pNN process composition and distributions



In [6]:
import scipy.interpolate as spi
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import mplhep
mplhep.set_style("CMS")
plt.rcParams["figure.figsize"] = (12.5,10)
def plotLimits2D(x,y, limits, ylabel, savename):
  bin_edges = []
#   mx = np.sort(np.unique(masses[:,0]))
#   my = np.sort(np.unique(masses[:,1]))
  mx = np.sort(np.unique(x))
  my = np.sort(np.unique(y))
  mx_edges = np.array([mx[0] - (mx[1]-mx[0])/2] + list(mx[:-1] + (mx[1:] - mx[:-1])/2) + [mx[-1] + (mx[-1]-mx[-2])/2])
  my_edges = np.array([my[0] - (my[1]-my[0])/2] + list(my[:-1] + (my[1:] - my[:-1])/2) + [my[-1] + (my[-1]-my[-2])/2])

#   spline = spi.interp2d(NMSSM_max_allowed_Y_gg.MX, NMSSM_max_allowed_Y_gg.MY, NMSSM_max_allowed_Y_gg.limit, kind='linear', fill_value=0)
#   max_allowed_values = [spline(m[0], m[1])[0] for m in masses]
#   for i, m in enumerate(masses):
#     print(m, max_allowed_values[i])
  
  #plt.hist2d(masses[:,0], masses[:,1], [mx_edges, my_edges], weights=limits[2], norm=matplotlib.colors.LogNorm())
  mx_edge_centers = (mx_edges[:-1]+mx_edges[1:])/2
  my_edge_centers = (my_edges[:-1]+my_edges[1:])/2
  interp_masses = []
  interp_limits = []
  for mxi in mx_edge_centers:
    for myi in my_edge_centers:
      interp_masses.append([mxi, myi])
  interp_masses = np.array(interp_masses)
  interp_limits = spi.griddata((x,y), limits, interp_masses, method="linear", fill_value=0)
  plt.hist2d(interp_masses[:,0], interp_masses[:,1], [mx_edges, my_edges], weights=interp_limits,norm=matplotlib.colors.Normalize(vmin=0.1))
  
  cbar = plt.colorbar()
  cbar.set_label(ylabel)
  plt.xlabel(r"$m_X$ [GeV]")
  plt.ylabel(r"$m_Y$ [GeV]")

#   plt.text(0.05, 0.9, r"$Y\rightarrow\tau\tau$", transform=plt.gca().transAxes, fontsize=32)
#   plt.text(0.05, 0.9, r"$Y\rightarrow\gamma\gamma$", transform=plt.gca().transAxes, fontsize=32)

  mplhep.cms.label(llabel="Work in Progress", data=True, lumi=132, loc=0)

  plt.savefig(savename+".png")
  plt.savefig(savename+".pdf")
  
#   plt.fill_between([250,650],[65,65],[my_edges[-1],my_edges[-1]],facecolor="none",hatch="/",edgecolor="red", label="Limit below maximally\nallowed in NMSSM")
#   plt.legend(frameon=True)
#   plt.savefig(savename+"_exclude.png")
#   plt.savefig(savename+"_exclude.pdf")
#   s = limits[2] < max_allowed_values
#   plt.scatter(masses[s,0], masses[s,1], marker='x', color="r", label="Limit below maximally allowed in NMSSM") 
#   plt.savefig(savename+"_exclude_points.png")
#   plt.savefig(savename+"_exclude_points.pdf")

  plt.clf()

/tmp/ipykernel_877253/1568446426.py:6: FutureWarning: ``set_style`` is deprecated: Naming convention is changing to match mpl. Use ``mplhep.style.use()``.
  mplhep.set_style("CMS")


In [7]:
limits=[]
limits1=[]
limits2=[]
x=[]
x1=[]
x2=[]
y=[]
y1=[]
y2=[]
z=[]
z1=[]
z2=[]
for d in limits_json:
    x.append(float(d["sig_proc"].split("_")[7]))
    y.append(float(d["sig_proc"].split("_")[9]))
    z.append(float(d["optimal_limit"]))
    limits.append([d["sig_proc"].split("_")[7], d["sig_proc"].split("_")[9],d["optimal_limit"]])
    # print (d["optimal_limit"],d["sig_proc"].split("_")[7], d["sig_proc"].split("_")[9])

for d in noMX_limits_json:
    x1.append(float(d["sig_proc"].split("_")[7]))
    y1.append(float(d["sig_proc"].split("_")[9]))
    z1.append(float(d["optimal_limit"]))
    limits1.append([d["sig_proc"].split("_")[7], d["sig_proc"].split("_")[9],d["optimal_limit"]])
    # print (d["optimal_limit"],d["sig_proc"].split("_")[7], d["sig_proc"].split("_")[9])
for d in noMX_DDGJ_limits_json:
    x2.append(float(d["sig_proc"].split("_")[7]))
    y2.append(float(d["sig_proc"].split("_")[9]))
    z2.append(float(d["optimal_limit"]))
    limits2.append([d["sig_proc"].split("_")[7], d["sig_proc"].split("_")[9],d["optimal_limit"]])

In [8]:
for l,l1,l2 in zip(limits,limits1,limits2):
    print ("mX ",l[0]," mY ",l[1], " old {} new {} ratio {}".format(l1[2],l2[2],round(l2[2]/l1[2],3)))

mX  1000  mY  100  old 0.18434524536132812 new 0.09698867797851562 ratio 0.526
mX  1000  mY  125  old 0.18491744995117188 new 0.04858970642089844 ratio 0.263
mX  1000  mY  150  old 0.17862319946289062 new 0.03483295440673828 ratio 0.195
mX  1000  mY  170  old 0.18434524536132812 new 0.028204917907714844 ratio 0.153
mX  1000  mY  190  old 0.18053054809570312 new 0.03077983856201172 ratio 0.17
mX  1000  mY  250  old 0.17862319946289062 new 0.05993843078613281 ratio 0.336
mX  1000  mY  300  old 0.18987655639648438 new 0.10786056518554688 ratio 0.568
mX  1000  mY  350  old 0.20008087158203125 new 0.11930465698242188 ratio 0.596
mX  1000  mY  400  old 0.20275115966796875 new 0.12884140014648438 ratio 0.635
mX  1000  mY  450  old 0.20503997802734375 new 0.13074874877929688 ratio 0.638
mX  1000  mY  500  old 0.19893646240234375 new 0.14390945434570312 ratio 0.723
mX  1000  mY  550  old 0.19588470458984375 new 0.14200210571289062 ratio 0.725
mX  1000  mY  600  old 0.18281936645507812 new 0.131

In [9]:
DDGJets_df = pd.read_parquet("/home/users/azecchin/Analysis/ResonantGGTT/Outputs/XtoYH_DDGJets/looper_merged_nominal_scored.parquet")
MCGJets_df = pd.read_parquet("/home/users/azecchin/Analysis/ResonantGGTT/Outputs/XtoYH_ggbb_2018_noMX_largeTraining/looper_merged_nominal_scored.parquet")

In [ ]:
DDGJets_df.fields

NameError: name 'DDGJets_df' is not defined